In [1]:
from ray.tune.schedulers import PopulationBasedTraining
from ray import tune, air
from ray.rllib.algorithms.ppo import PPO, PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.algorithms.ppo import PPO, PPOConfig
# TODO relative import
from envs import MultiAgentBuildingEnv, MonoAgentBuildingEnv

E0000 00:00:1732186575.123394   55442 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732186575.126703   55442 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
pbt = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=4,
    resample_probability=0.25,
    hyperparam_mutations={
        "lr": tune.uniform(1e-5, 0.1),
        "batch_mode": ["complete_episodes"],
        #"train_batch_size": [4000],
        "sgd_minibatch_size": [32, 64, 128, 256, 512],
        "num_sgd_iter": [10, 20, 30],
        "clip_param": tune.uniform(0.1, 0.3),
    },
)

tuner = tune.Tuner(
    PPO,
    param_space=(
        MultiAgentBuildingEnv.get_algo_config(
            PPOConfig()
            .rollouts(
                num_rollout_workers=2, 
                rollout_fragment_length='auto',
                # rollout_fragment_length=200,
            )
            .resources(num_gpus=0.25)
        )
    ),
    tune_config=tune.TuneConfig(
        #reuse_actors=True,
        scheduler=pbt,
        num_samples=2,
        metric="env_runners/episode_reward_mean",
        mode="max",
    ),
    run_config=air.RunConfig(
        stop={"training_iteration": 200},
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True
        ),
        verbose=2,
    )
)
results = tuner.fit()

2024-11-21 10:56:21,546	WARNING deprecation.py:50 -- DeprecationWarning: `rollouts` has been deprecated. Use `AlgorithmConfig.env_runners(..)` instead. This will raise an error in the future!
2024-11-21 10:56:21,547	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1362: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-11-21_10-56-21_571496_55442/logs/gcs_server.out' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1362: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-11-21_10-56-21_571496_55442/logs/gcs_server.err' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/

(pid=59570) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=59570) E0000 00:00:1732186583.559695   59570 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=59570) E0000 00:00:1732186583.563092   59570 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=60007) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(pid=60007) E0000 00:00:1732186590.287209   60007 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

Trial name,agent_timesteps_total,counters,custom_metrics,env_runners,episode_media,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_MultiAgentBuildingEnv_3ef84_00000,256000,"{'num_env_steps_sampled': 32000, 'num_env_steps_trained': 32000, 'num_agent_steps_sampled': 256000, 'num_agent_steps_trained': 256000}",{},"{'episode_reward_max': 2.1147200478185884, 'episode_reward_min': -0.32585442486328087, 'episode_reward_mean': 0.45286044013029064, 'episode_len_mean': 4608.0, 'episode_media': {}, 'episodes_timesteps_total': 27648, 'policy_reward_min': {'0FWEST': -0.17440255020157552, '1FEAST1': -0.32289497425075003, '1FWEST': -0.04005226177612836, '0FEAST1': -0.03321275610167676, '1FWEST1': -0.3653213811219444, '0FEAST': -0.2594869260813714, '1FEAST': -0.23120265483390454, '0FWEST1': -0.06106112306702458}, 'policy_reward_max': {'0FWEST': 0.3606833049139906, '1FEAST1': 0.3889675761614517, '1FWEST': 0.2616883555478743, '0FEAST1': 0.19097767742921828, '1FWEST1': 0.4455361186563782, '0FEAST': 0.45769545266623535, '1FEAST': 0.2616883555478786, '0FWEST1': 0.13819309162214036}, 'policy_reward_mean': {'0FWEST': 0.06986580716353484, '1FEAST1': 0.005047685554767048, '1FWEST': 0.06750878455957966, '0FEAST1': 0.08754347669319763, '1FWEST1': 0.04629558112398286, '0FEAST': 0.08282943148528735, '1FEAST': 0.027439400292344544, '0FWEST1': 0.0663302732576008}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [1.547314345582119, 2.1147200478185884, -0.0682525237925512, -0.2567784839022712, -0.32585442486328087, -0.29398632006086023], 'episode_lengths': [4608, 4608, 4608, 4608, 4608, 4608], 'policy_0FWEST_reward': [0.15363953675602413, 0.3606833049139906, 0.16647736286960207, -0.17440255020157552, -0.10348752993821514, 0.016284718581382876], 'policy_1FEAST1_reward': [0.3233451642407964, 0.3889675761614517, -0.20828923115927211, -0.32289497425075003, -0.23783781836365778, 0.08699539670003409], 'policy_1FWEST_reward': [0.054644587389907906, 0.2616883555478743, -0.0032282646151708186, -0.04005226177612836, 0.15814197910080535, -0.02614168828981034], 'policy_0FEAST1_reward': [0.11121312988482843, 0.19097767742921828, 0.0321270744441608, 0.08722695883744644, 0.1369287756652086, -0.03321275610167676], 'policy_1FWEST1_reward': [0.08292885863736746, 0.4455361186563782, -0.03151253586263155, -0.3653213811219444, -0.01863471619583079, 0.16477714263055826], 'policy_0FEAST_reward': [0.45769545266623535, 0.08491166025123714, -0.13757855304061328, 0.41956714599512374, -0.06813219087888758, -0.2594869260813714], 'policy_1FEAST_reward': [0.25263448612214234, 0.2616883555478786, -0.0244414680507634, 0.037729484154389765, -0.13177180118567558, -0.23120265483390454], 'policy_0FWEST1_reward': [0.11121312988482693, 0.1202669993105652, 0.13819309162214036, 0.10136909446117776, -0.06106112306702458, -0.011999552666080868]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.8876206749837818, 'mean_inference_ms': 5.084177636763962, 'mean_action_processing_ms': 0.5554730738967234, 'mean_env_wait_ms': 9.100340754405595, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {}, 'num_episodes': 0, 'episode_return_max': 2.1147200478185884, 'episode_return_min': -0.32585442486328087, 'episode_return_mean': 0.45286044013029064, 'episodes_this_iter': 0}",{},256000,256000,256000,32000,32000,4000,60.2883,32000,4000,60.2883,2,0,0,4000,"{'cpu_util_percent': 29.69574468085106, 'ram_util_percent': 41.33510638297873}","{'training_iteration_time_ms': 69034.482, 'restore_workers_time_ms': 0.013, 'training_step_time_ms': 69034.446, 'sample_time_ms': 30312.978, 'learn_time_ms': 38714.527, 'learn_through

(PPO pid=59575) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/AD/user/ray_results/PPO_2024-11-21_10-56-21/PPO_MultiAgentBuildingEnv_3ef84_00001_1_2024-11-21_10-56-22/checkpoint_000000)
(PPO pid=59575) 2024-11-21 10:57:18,356	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!
2024-11-21 11:05:50,498	INFO pbt.py:878 -- 

[PopulationBasedTraining] [Exploit] Cloning trial 3ef84_00001 (score = 2.106981) into trial 3ef84_00000 (score = 0.452860)

2024-11-21 11:05:50,500	INFO pbt.py:905 -- 

[PopulationBasedTraining] [Explore] Perturbed the hyperparameter config of trial3ef84_00000:
batch_mode : truncate_episodes --- (resample) --> complete_episodes
lr : 5e-05 --- (resample) --> 0.07795170850835062
sgd_minibatch_size : 128 --- (resample) --> 512
num_sgd_iter : 30 --- (shift left) --> 20
clip_param : 0.3 --- (* 1.2) --> 0.36

(pid=63900) WARNING: All log messages before absl::Initializ

In [ ]:
best_result = results.get_best_result()
print("Best Hyperparameters found: ", best_result)

2024-11-16 09:19:59,228	WARNING experiment_analysis.py:558 -- Could not find best trial. Did you pass the correct `metric` parameter?


  0%|          | 0.0/100 [00:10<00:02, 35.10it/s, Warming up]


RuntimeError: No best trial found for the given metric: env_runners/episode_reward_mean. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

In [ ]:
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from controllables.core import BaseVariable
from controllables.core.tools.records import VariableRecords
from ray.rllib.algorithms.ppo import PPO, PPOConfig
from ray.rllib.algorithms.callbacks import DefaultCallbacks

from controllables.core import BaseVariable, ComputedVariable
from controllables.core.tools.records import VariableRecords

import pythermalcomfort as pytc
import numpy as _numpy_


from controllables.energyplus import Actuator, OutputVariable, OutputMeter

class PMVVariable(BaseVariable):
    def __init__(
        self, 
        tdb: BaseVariable,
        tr: BaseVariable,
        rh: BaseVariable,
        metab_rate=1.5, clothing=.5, pmv_limit=.5,
    ):
        self.tdb = tdb
        self.tr = tr
        self.rh = rh
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
    
    @property
    def value(self):
        res = pytc.models.pmv_ppd(
            tdb=self.tdb.value, 
            tr=self.tr.value, 
            # calculate relative air speed
            vr=pytc.utilities.v_relative(v=0.1, met=self._metab_rate), 
            rh=self.rh.value, 
            met=self._metab_rate, 
            # calculate dynamic clothing
            clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
            limit_inputs=False,
        )['pmv']

        return res

class PlottingCallbacks(DefaultCallbacks):
    def __init__(self):
        self.env_records: dict[object, VariableRecords] = None

    def on_episode_start(self, *, episode, worker, **kwargs):
        env: MultiAgentBuildingEnv = worker.env
        system = env.system
        if self.env_records is None:
            system.add('logging:progress')
            self.env_records = dict()
            for agent_ref in env.agents:
                if agent_ref == 'CHILLER' or agent_ref == 'AHU':
                    # self.env_records[agent_ref] = records = VariableRecords({
                    # 'AHU COOLING COIL': system[OutputVariable.Ref('Cooling Coil Total Cooling Rate', 'AIR LOOP AHU COOLING COIL')],
                    # 'Electricity': system[OutputMeter.Ref('Electricity:HVAC')]
                    # }, maxlen=10_000)
                    pass
                else:
                    tdb = env.agents[agent_ref].observation['temperature:drybulb']
                    tr = env.agents[agent_ref].observation['temperature:radiant']
                    rh = env.agents[agent_ref].observation['humidity']
                    pmv = PMVVariable(tdb=tdb, tr=tr, rh=rh)
                    self.env_records[agent_ref] = records = VariableRecords({
                        '🕰️': system['time'],
                        '🍩': env.agents[agent_ref].reward,
                        'pmv': pmv,
                        'occupancy': env.agents[agent_ref].observation['Office Occupancy'],
                        # 'tstat': env.agents[agent_ref].action['thermostat'],
                        'temp': tdb,
                        # 'AHU COOLING COIL': system[OutputVariable.Ref('Cooling Coil Total Cooling Rate', 'AIR LOOP AHU COOLING COIL')],
                        'elec': system[OutputMeter.Ref('Electricity:HVAC')],
                    }, maxlen=10_000)
                    # display(
                    #     records.plot.scatter(x='🕰️', y='pv', label=repr(agent_ref))
                    #     .watch(records.events['change'] % 1_000)
                    # )

    def on_episode_step(self, *, episode, **kwargs):
        for _, env_records in self.env_records.items():
            env_records.poll()

    def on_episode_end(self, *, episode, **kwargs):
        for agent_ref, env_records in self.env_records.items():
            env_records.dataframe().to_csv(f'datasave/20241113/records_train_slope_angle_distant_elc_changed-{agent_ref}.csv', index=False)


In [ ]:
config_eval = (
    MonoAgentBuildingEnv.get_algo_config(
        PPOConfig()
        .rollouts(
            num_rollout_workers=4, 
            rollout_fragment_length='auto',
        )
        .resources(num_gpus=1/2)
    )
    .env_runners(
        num_env_runners=0,
        create_env_on_local_worker=True,
    )
    .evaluation(
        evaluation_duration=1,
        evaluation_duration_unit='episodes',
        #evaluation_interval=1,
        evaluation_num_env_runners=0,
    )
    .callbacks(PlottingCallbacks)
)

algo_eval = PPO(config_eval)
#algo_eval.restore(best_result.checkpoint)
algo_eval.restore('/home/AD/user/ray_results/PPO_2024-11-14_17-33-56/PPO_MonoAgentBuildingEnv_a0b84_00001_1_2024-11-14_17-33-57/checkpoint_000049')
#algo_eval.restore("/home/AD/user/ray_results/PPO_2024-10-18_15-32-34/PPO_UserMultiAgentEnv_331fb_00000_0_2024-10-18_15-32-35/checkpoint_000013")

2024-11-15 05:44:55,191	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json

/usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_ptv97bl8'>
  _warnings.warn(warn_message, ResourceWarning)
/usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_z9131ogb'>
  _warnings.warn(warn_message, ResourceWarning)
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Air Relative Humidity', key='0FGROUNDFLOOREAST1:OPENOFFICE')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
  _warnings_.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Mean Air T

In [ ]:
for _ in range(2):
    algo_eval.evaluate()

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Air Relative Humidity', key='1FFIRSTFLOOREAST1:OPENOFFICE')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
  _warnings_.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Mean Air Temperature', key='1FFIRSTFLOOREAST1:OPENOFFICE')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
  _warnings_.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/vari

  0%|          | 0/100 [00:00<?, ?it/s]

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/core/tools/gymnasium/spaces.py:224: RuntimeWarning: overflow encountered in cast
  return _numpy_.array(
